In [1]:
!pip install --quiet altair
!pip install --quiet tabulate


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import altair as alt
import pandas as pd

from tabulate import tabulate

In [31]:
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [3]:
click_df = pd.read_parquet("output/clicks.parquet", columns=["query", "title", "abstract", "url_md5", "position", "click"])
sample_df = pd.read_parquet("output/samples.parquet", columns=["query", "title", "abstract", "position", "click"])
annotation_df = pd.read_parquet("output/annotations.parquet")

## Utils

In [20]:
def print_table(stats):
    stats_table = [[k, f"{v:,}"] for k, v in stats.items()]
    print(tabulate(stats_table, headers=["Statistic", "Value"], tablefmt="pretty"))

def match(df1, df2, on, left_columns=[], right_columns=[], unique=False):
    
    if unique:
        df1 = df1[on + left_columns].drop_duplicates(on)
        df2 = df2[on + right_columns].drop_duplicates(on)

    return df1[on + left_columns].merge(df2[on + right_columns], on=on)

def missing_text(df, column, missing_token="21429"):
    return (df[column] == missing_token).agg(["sum", "mean"]).round(decimals=4).to_dict()

def count_stats(df, groupby, unique = False):
    groupby = groupby if type(groupby) is list else [groupby]
    
    if unique:
        df = df.drop_duplicates(["query", "title"] + groupby)
    
    source = df.groupby(groupby).agg(
        documents=("title", "count"),
    ).reset_index()

    source["perc_documents"] = (source.documents / source.documents.sum()).round(4)
    return source

## Part 1: Expert annotations


#### TL;DR
- **Query ids are not unique**: The expert annotation dataset uses the same query_id to refer to queries with different text.
- **Not all test queries were created as described in the paper**: The median query has 54 documents in the test set. However, there are 63 queries (0.89%) with less then 10 documents.
- **Query-document pairs in the test set are largely not redundant:** Overall, the test set contains 397K query-document pairs, and 368K unique query document pairs by text. Meaning, 7.2% of query-document pairs contain duplicate text.
- **No missing text:** Not a single query-doc pair misses text in its title or abstract in the test set, which contrasts the high rates of missing text in the click dataset
- **Frequent queries have more relevant annotated docs**: More popular queries contain more relevant documents. While 33.1% of documents in the most frequent queries are non-relevant (0), 75.4% of documents in the least frequent queries are not relevant.
- **Frequent queries have more navigational queries**: Documents with relevance label 4 are the destination page of navigational queries. 320 queries (4.56%) are navigational queries. The more frequent the query, the higher the chance of being a navigational query. 16% of the most popular queries are navigational, while only 0.2% of the least frequent queries are navigational.
- **Navigational queries can have many destinations**: The average navigational query has two destination. Two queries have more than 10 destination pages (meaning more than 10 documents rated with 4).

In [5]:
annotation_df["is_navigational"] = annotation_df["label"] == 4

query_df = annotation_df.groupby(["query_no", "frequency_bucket"]).agg(
    navigational_query=("is_navigational", "max"),
    navigational_documents=("is_navigational", "sum"),
).reset_index()

### 1.1 Base annotation statistics

In [6]:
print(f"Unique queries (by text): {annotation_df['query'].nunique():,}")
print(f"Unique queries (by id): {annotation_df['query_id'].nunique():,}")
print(f"Query-doc pairs: {len(annotation_df):,}")
print(f"Unique query/title comb.: {len(annotation_df[['query_id', 'title']].drop_duplicates()):,}")
print()
print(f"Unique titles per query:\n{annotation_df.groupby('query')['title'].nunique().describe(percentiles=[0.5])}\n")

Unique queries (by text): 7,008
Unique queries (by id): 5,201
Query-doc pairs: 397,572
Unique query/title comb.: 368,792

Unique titles per query:
count    7008.000000
mean       52.628282
std         9.313274
min         1.000000
50%        54.000000
max       109.000000
Name: title, dtype: float64



### 1.2 Relevace distribution

In [7]:
source = count_stats(annotation_df, groupby=["label"])

alt.Chart(
    source,
    title="Annotation distribution over documents",
    width=600,
    height=250,
).mark_bar().encode(
    x=alt.X("label:O", title="Relevance").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of documents with relevance").axis(format="%"),
    color=alt.Color("label:N", title="Relevance"),
    tooltip=list(source.columns),
)

alt.Chart(...)

In [8]:
source = count_stats(annotation_df, groupby=["frequency_bucket", "label"])

alt.Chart(
    source,
    title="Annotation distribution over query frequency",
    width=600,
    height=250
).mark_bar().encode(
    x=alt.X("frequency_bucket:O", title="Query frequency (high to low)").axis(labelAngle=0),
    y=alt.Y("documents", title="% of documents with relevance").stack("normalize"),
    color=alt.Color("label:N", title="Relevance"),
    tooltip=list(source.columns),
)

alt.Chart(...)

### 1.3 Navigational queries

In [9]:
print(f"Navigational queries: {query_df['navigational_query'].sum()}, {query_df['navigational_query'].mean():.2%}")
print()
print(f"Destination documents:\n{query_df[query_df['navigational_query']]['navigational_documents'].describe(percentiles=[0.5, 0.9])}")

Navigational queries: 320, 4.56%

Destination documents:
count    320.000000
mean       2.231250
std        1.948303
min        1.000000
50%        2.000000
90%        4.000000
max       21.000000
Name: navigational_documents, dtype: float64


In [10]:
source = query_df.groupby(["frequency_bucket"]).agg(navigational_query=("navigational_query", "mean")).reset_index()

alt.Chart(source, title="Navigational queries and query frequency",  width=600, height=250).mark_bar().encode(
    x=alt.X("frequency_bucket:O", title="Query frequency (high to low)").axis(labelAngle=0),
    y=alt.Y("navigational_query", title="Rate of navigational queries").axis(format="%"),
    tooltip=list(source.columns),
)

alt.Chart(...)

### 1.4 Text analysis

In [11]:
print(f"Missing query: {missing_text(annotation_df, column='query')}")
print(f"Missing title: {missing_text(annotation_df, column='title')}")
print(f"Missing abstract: {missing_text(annotation_df, column='abstract')}")

Missing query: {'sum': 0.0, 'mean': 0.0}
Missing title: {'sum': 0.0, 'mean': 0.0}
Missing abstract: {'sum': 0.0, 'mean': 0.0}


## Part 2: Click dataset

#### TL;DR
- **Query-doc pairs are highly redundant**: The 59.9M query-document pairs contain 106k unique query-document texts. Meaning 99.8% of pairs are repeating. Note, that not all query-doc pairs from the test query appear in the click dataset.
- **Some queries rank a high number of different documents**: The median query has 22 unique urls, 19 text pairs. However, there is at least one query for which 28,439 unique urls occur.
- **Missing text is prevalent and more prevalent compared to the overall corpus**: 16.3% of document titles are missing (13% in partition-0) and 94.1% of abstracts are missing (84.8% in partition-0).
- **More clicks for queries that appear in the test set**: 92.4% of filtered queries contain at least one click, while only 61.39% of clicks in partition-0 of Baidu-ULTR.

### 2.1 Click statistics

In [12]:
print(f"Unique queries (by text): {click_df['query'].nunique():,}")
print(f"Query-doc pairs: {len(click_df):,}")
print(f"Unique query/title comb.: {len(click_df[['query', 'title']].drop_duplicates()):,}")
print()
print(f"Unique urls per query:\n{click_df.groupby('query')['url_md5'].nunique().describe(percentiles=[0.5])}")
print()
print(f"Unique titles per query:\n{click_df.groupby('query')['title'].nunique().describe(percentiles=[0.5])}")
print()
print(f"Queries that contain at least one click: {click_df.groupby(['query'])['click'].max().mean():.2%}, in Baidu-ULTR: {sample_df.groupby(['query'])['click'].max().mean():.2%}")

Unique queries (by text): 3,366
Query-doc pairs: 59,891,217
Unique query/title comb.: 106,092

Unique urls per query:
count     3366.000000
mean       112.923648
std        947.676248
min          2.000000
50%         22.000000
max      28439.000000
Name: url_md5, dtype: float64

Unique titles per query:
count    3366.000000
mean       31.518717
std       116.702019
min         2.000000
50%        19.000000
max      4335.000000
Name: title, dtype: float64

Queries that contain at least one click: 92.36%, in Baidu-ULTR: 61.39%


In [13]:
source = click_df.groupby(['position'])['click'].mean().reset_index()
source2 = sample_df.groupby(['position'])['click'].mean().reset_index()

alt.Chart(source, title="CTR per rank (filtered subset)",  width=600, height=150).mark_bar().encode(
    x=alt.X("position:O", title="Position").axis(labelAngle=0),
    y=alt.Y("click", title="CTR").axis(format="%"),
    tooltip=list(source.columns),
) & alt.Chart(source2, title="CTR per rank (Baidu-ULTR partition 0)",  width=600, height=150).mark_bar().encode(
    x=alt.X("position:O", title="Position").axis(labelAngle=0),
    y=alt.Y("click", title="CTR").axis(format="%"),
    tooltip=list(source.columns),
)

alt.VConcatChart(...)

### 2.2 Missing text

In [14]:
print(f"Missing query in filtered subset: {missing_text(click_df, column='query')}, overall in Baidu-ULTR: {missing_text(sample_df, column='query')}")
print(f"Missing title in filtered subset: {missing_text(click_df, column='title')}, overall in Baidu-ULTR: {missing_text(sample_df, column='title')}")
print(f"Missing abstract: {missing_text(click_df, column='abstract')}, overall in Baidu-ULTR: {missing_text(sample_df, column='abstract')}")

Missing query in filtered subset: {'sum': 0.0, 'mean': 0.0}, overall in Baidu-ULTR: {'sum': 20.0, 'mean': 0.0}
Missing title in filtered subset: {'sum': 9744019.0, 'mean': 0.1627}, overall in Baidu-ULTR: {'sum': 816667.0, 'mean': 0.1302}
Missing abstract: {'sum': 56333874.0, 'mean': 0.9406}, overall in Baidu-ULTR: {'sum': 5318177.0, 'mean': 0.848}


## Part 3: Match between clicks and annotations:

In [15]:
query_df = match(annotation_df, click_df, on=['query'], unique=True)

print(f"Queries in both datasets: {len(query_df):,}")
print(f"% of annotated queries in both datasets: % of all test queries: {len(query_df) / annotation_df['query'].nunique():.4f}")
print()
print(f"Query/title comb. in both datasets: {len(match(annotation_df, click_df, on=['query', 'title'])):,}")
print(f"Unique query/title comb. in both datasets: {len(match(annotation_df, click_df, on=['query', 'title'], unique=True)):,}")
print(f"% of query/title comb. of the test set occur in both datasets: {len(match(annotation_df, click_df, on=['query', 'title'], unique=True)) / len(annotation_df[['query', 'title']].drop_duplicates()):.4f}")
print()
print(f"Query/title/abstract comb. in both datasets: {len(match(annotation_df, click_df, on=['query', 'title', 'abstract'])):,}")

Queries in both datasets: 3,366
% of annotated queries in both datasets: % of all test queries: 0.4803

Query/title comb. in both datasets: 51,746,434
Unique query/title comb. in both datasets: 8,672
% of query/title comb. of the test set occur in both datasets: 0.0235

Query/title/abstract comb. in both datasets: 0


## Part 4: Focusing on query/title matches

In [16]:
match_df = match(annotation_df, click_df, on=['query', 'title'], left_columns=["frequency_bucket", "label"], right_columns=["position", "click"])

print(f"Queries that have at least one matching document (by title): {match_df['query'].nunique():,}")
print(f"% of queries that occur in the click dataset AND have matching documents: {match_df['query'].nunique() / annotation_df['query'].nunique():.4f}")
print()
print(f"Unique documents per query:\n{match_df.groupby('query')['title'].nunique().describe(percentiles=[0.5, 0.9]).apply(lambda x: format(x, '.2f'))}")
print()
print(f"Documents per query:\n{match_df.groupby('query').size().describe(percentiles=[0.5, 0.9]).apply(lambda x: format(x, '.2f'))}")

Queries that have at least one matching document (by title): 2,744
% of queries that occur in the click dataset AND have matching documents: 0.3916

Unique documents per query:
count    2744.00
mean        3.16
std         2.08
min         1.00
50%         3.00
90%         6.00
max        19.00
Name: title, dtype: object

Documents per query:
count        2744.00
mean        18858.03
std        621974.72
min             1.00
50%            67.00
90%          5966.60
max      32394427.00
dtype: object


In [17]:
source = match_df.groupby("query").agg(total_documents=("title", "count"), unique_documents=("title", "nunique")).reset_index()

base = alt.Chart(source).transform_calculate(
    log_total_documents = 'log(datum.total_documents)/log(10)'
)

base.mark_bar().encode(
    x=alt.X("unique_documents:O", title="# of unique docs per query"),
    y=alt.Y("count(query):Q", title="# of queries"),
    tooltip=["unique_documents:O", "count(query):Q"],
) | base.mark_bar().encode(
    x=alt.X("log_total_documents:Q", title="# of total docs per query (log scale)").bin().axis(labelExpr='format(pow(10, datum.value), ",")', labelAngle=-45, labelOverlap=False),
    y=alt.Y("count(query):Q", title="# of queries"),
    tooltip=alt.Tooltip(["count(query):Q"], bin=True),
)

alt.HConcatChart(...)

#### Relevance distribution shift

In [19]:
alt.Chart(
    count_stats(annotation_df, groupby="label", unique=True),
    title="Expert Annotations",
    width=400,
    height=150,
).mark_bar().encode(
    x=alt.X("label:O", title="Relevance label").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of (unique) query/title pairs").axis(format="%"),
    tooltip=["perc_documents"],
) & alt.Chart(
    count_stats(match_df, groupby="label", unique=True),
    title="Expert Annotations + Clicks",
    width=400,
    height=150,
).mark_bar().encode(
    x=alt.X("label:O", title="Relevance label").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of (unique) query/title pairs").axis(format="%"),
    tooltip=["perc_documents"],
)

alt.VConcatChart(...)

#### Query popularity shift

In [21]:
alt.Chart(
    count_stats(annotation_df, groupby="frequency_bucket", unique=True),
    title="Expert Annotations",
    width=600,
    height=150,
).mark_bar().encode(
    x=alt.X("frequency_bucket:O", title="Query frequency (high to low)").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of (unique) query/title pairs").axis(format="%"),
    tooltip=["perc_documents"],
) & alt.Chart(
    count_stats(match_df, groupby="frequency_bucket", unique=True),
    title="Expert Annotations + Clicks",
    width=600,
    height=150,
).mark_bar().encode(
    x=alt.X("frequency_bucket:O", title="Query frequency (high to low)").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of (unique) query/title pairs").axis(format="%"),
    tooltip=["perc_documents"],
)

alt.VConcatChart(...)

#### Doc positions shift

In [22]:
alt.Chart(
    count_stats(sample_df, groupby="position", unique=True),
    title="Clicks",
    width=600,
    height=150,
).mark_bar().encode(
    x=alt.X("position:O", title="Query frequency (high to low)").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of (unique) query/title pairs").axis(format="%"),
    tooltip=["perc_documents"],
) & alt.Chart(
    count_stats(match_df, groupby="position", unique=True),
    title="Expert Annotations + Clicks",
    width=600,
    height=150,
).mark_bar().encode(
    x=alt.X("position:O", title="Query frequency (high to low)").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of (unique) query/title pairs").axis(format="%"),
    tooltip=["perc_documents"],
)

alt.VConcatChart(...)

## Part 5: Focusing on queries that have matches in consecutive positions

In [23]:
def longest_consecutive_subsequence(positions):
    positions = sorted(positions)
    longest_streak = []
    current_streak = [positions[0]]
    
    for i in range(1, len(positions)):
        if positions[i] == positions[i - 1] + 1:
            current_streak.append(positions[i])
        else:
            if len(current_streak) > len(longest_streak):
                longest_streak = current_streak
            current_streak = [positions[i]]
    
    if len(current_streak) > len(longest_streak):
        longest_streak = current_streak
        
    return longest_streak

position_df = match_df.drop_duplicates(["query", "position"]).groupby('query').agg(positions=("position", longest_consecutive_subsequence)).reset_index()
position_df["min_position"] = position_df["positions"].map(lambda x: x[0])
position_df["max_position"] = position_df["positions"].map(lambda x: x[-1])
position_df["consecutive_positions"] = position_df["positions"].map(len)
position_df.head()

,query,positions,min_position,max_position,consecutive_positions
0,10016 2638 21326 21355 19701 5924 19701 18483 ...,"[7, 8, 9, 10]",7,10,4
1,10016 2638 9066 15533 9763 8207 16556 19675 10...,"[7, 8]",7,8,2
2,10048 8815 12496 19165 13933 9771 13933 6145 6...,[2],2,2,1
3,10059 12307 7897 9102 241 11676,"[4, 5, 6, 7, 8]",4,8,5
4,10059 16260 3742 3973 15425 20812,"[3, 4, 5, 6, 7, 8, 9, 10, 11]",3,11,9


In [92]:
top_df = position_df[(position_df.consecutive_positions >= 5)]
top_df = match_df[match_df["query"].isin(top_df["query"])]

print(f"Queries with at documents in at least 3 consecutive positions: {len(position_df[position_df.consecutive_positions >= 3]):,}")
print(f"Queries with at documents in at least 5 consecutive positions: {len(position_df[position_df.consecutive_positions >= 5]):,}")
print(f"Queries with at documents in at least 10 consecutive positions: {len(position_df[position_df.consecutive_positions >= 10]):,}")
print(f"Queries with at documents in at least 10 consecutive positions starting from 1: {len(position_df[(position_df.consecutive_positions >= 10) & position_df.min_position == 1]):,}")

Queries with at documents in at least 3 consecutive positions: 1,813
Queries with at documents in at least 5 consecutive positions: 1,242
Queries with at documents in at least 10 consecutive positions: 437
Queries with at documents in at least 10 consecutive positions starting from 1: 336


In [94]:
alt.Chart(
    position_df,
    title="Clicks",
    width=600,
    height=150,
).mark_bar().encode(
    x=alt.X("consecutive_positions:O", title="# of documents in consecutive positions").axis(labelAngle=0),
    y=alt.Y("count(query)", title="# of queries"),
)

alt.Chart(...)

In [85]:
alt.Chart(
    count_stats(top_df, groupby="frequency_bucket", unique=True),
    title="Expert Annotations + Clicks",
    width=600,
    height=150,
).mark_bar().encode(
    x=alt.X("frequency_bucket:O", title="Query frequency (high to low)").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of (unique) query/title pairs").axis(format="%"),
    tooltip=["perc_documents"],
) & alt.Chart(
    count_stats(top_df, groupby="label", unique=True),
    title="Expert Annotations + Clicks",
    width=400,
    height=150,
).mark_bar().encode(
    x=alt.X("label:O", title="Relevance label").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of (unique) query/title pairs").axis(format="%"),
    tooltip=["perc_documents"],
)

alt.VConcatChart(...)

## Part 6: User-annotator agreement on top 5 subset
Trust bias?!?! and cascade?

In [100]:
source = top_df.groupby(["query", "position", "title"]).agg(
    ctr=("click", "mean"),
    clicks=("click", "sum"),
    impressions=("click", "count"),
    label=("label", "max"),
).reset_index()

(alt.Chart(source, title="Avg. CTR", width=250).mark_rect().encode(
    x="label:O",
    y="position:O",
    color="mean(ctr)",
    tooltip=["mean(ctr)", "sum(impressions)", "count(title)"],
) | alt.Chart(source, title="# of queries", width=250).mark_rect().encode(
    x="label:O",
    y="position:O",
    color=alt.Color("count(query)"),
    tooltip=["count(query)"]
)| alt.Chart(source, title="# of clicks", width=250).mark_rect().encode(
    x="label:O",
    y="position:O",
    color=alt.Color("sum(clicks)"),
) | alt.Chart(source, title="# of impressions", width=250).mark_rect().encode(
    x="label:O",
    y="position:O",
    color=alt.Color("sum(impressions)").scale(type="log"),
    tooltip=["sum(impressions)"]
)).resolve_scale(color="independent")

alt.HConcatChart(...)

In [101]:
selection = alt.selection_point(fields=["label"], bind="legend")

base = alt.Chart(source, title="Avg. CTR per position and expert label", width=800).encode(
    x=alt.X("position:O", title="Position"),
    color=alt.Color("label:N", title="Relevance"),
    tooltip=["mean(ctr)", "sum(impressions)", "count(title)"],
).add_params(selection)

mean = base.mark_line(point=True).encode(
    y=alt.Y("mean(ctr)", title="CTR"),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    strokeWidth=alt.value(2),
)

ci = base.mark_errorband().encode(
    y=alt.Y("ctr", title="CTR"),
    opacity=alt.condition(selection, alt.value(0.2), alt.value(0.05)),
)

impressions = alt.Chart(source, title="Queries", width=800).mark_bar().encode(
    x=alt.X("position:O", title="Position"),
    xOffset="label:O",
    y=alt.Y("count(query)", title="# of queries").scale(type="log"),
    color=alt.Color("label:N", title="Relevance"),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    tooltip=["count(query)", "count(title)"],
)

(ci + mean) & impressions

alt.VConcatChart(...)